In [1]:
import os
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from shapely import wkt
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# ============================================================================
# CONFIGURATION
# ============================================================================
print("=" * 80)
print("FUSION SPATIALE : FIRE+SOIL+LANDCOVER (DISTANCE MINIMALE POUR POINTS ORPHELINS)")
print("=" * 80)

# Chemins des fichiers
base_path = r"C:\Users\pc\Desktop\DM\datasets"
fire_soil_path = os.path.join(base_path, "merged_fire_soil", "fire_soil_merged_knn_idw.csv")
landcover_path = os.path.join(base_path, "landcover", "landcover_merged_encoded.csv")
output_dir = os.path.join(base_path, "merged_fire_soil_landcover_distance_min")

# Créer le dossier de sortie
os.makedirs(output_dir, exist_ok=True)

# ============================================================================
# ÉTAPE 1 : CHARGER LES DONNÉES
# ============================================================================
print("\n[1/8] Chargement des données...")

print("   📍 Chargement du dataset FIRE+SOIL...")
df_fire_soil = pd.read_csv(fire_soil_path)
print(f"      ✓ Fire+Soil chargé : {len(df_fire_soil):,} points")

print("\n   🌳 Chargement du dataset LANDCOVER...")
df_landcover = pd.read_csv(landcover_path)
print(f"      ✓ Landcover chargé : {len(df_landcover):,} polygones")

# ============================================================================
# ÉTAPE 2 : CONVERTIR EN GEODATAFRAMES
# ============================================================================
print("\n[2/8] Conversion en GeoDataFrames...")

# Fire+Soil → Points
print("   📍 Création des points FIRE+SOIL...")
geometry_fire_soil = [Point(xy) for xy in zip(df_fire_soil['longitude'], df_fire_soil['latitude'])]
gdf_fire_soil = gpd.GeoDataFrame(df_fire_soil, geometry=geometry_fire_soil, crs='EPSG:4326')
gdf_fire_soil['point_index'] = gdf_fire_soil.index  # Garder l'index original
print(f"      ✓ {len(gdf_fire_soil):,} points créés")

# Landcover → Polygones
print("   🌳 Conversion des polygones LANDCOVER...")
df_landcover['geometry'] = df_landcover['geometry'].apply(wkt.loads)
gdf_landcover = gpd.GeoDataFrame(df_landcover, geometry='geometry', crs='EPSG:4326')
print(f"      ✓ {len(gdf_landcover):,} polygones créés")

# ============================================================================
# ÉTAPE 3 : JOINTURE SPATIALE (POINTS DANS POLYGONES)
# ============================================================================
print("\n[3/8] Jointure spatiale (intersects) pour points dans polygones...")

gdf_matched = gpd.sjoin(
    gdf_fire_soil,
    gdf_landcover,
    how='inner',
    predicate='intersects'
)

print(f"      ✓ Points matchés : {len(gdf_matched):,} correspondances")

# ============================================================================
# ÉTAPE 4 : GESTION DES DOUBLONS
# ============================================================================
print("\n[4/8] Gestion des doublons...")

duplicated = gdf_matched[gdf_matched.duplicated(subset=['point_index'], keep=False)]
if len(duplicated) > 0:
    n_unique_dups = duplicated['point_index'].nunique()
    print(f"   ⚠️  {n_unique_dups:,} points dans plusieurs polygones")
    print(f"   🔧 Application : keep='first'")
    gdf_matched = gdf_matched.drop_duplicates(subset=['point_index'], keep='first')
    print(f"      ✓ {len(gdf_matched):,} points uniques après dédoublonnage")
else:
    print(f"   ✅ Aucun doublon détecté")

# ============================================================================
# ÉTAPE 5 : IDENTIFIER LES POINTS ORPHELINS (SANS POLYGONE)
# ============================================================================
print("\n[5/8] Identification des points orphelins...")

matched_indices = set(gdf_matched['point_index'].values)
orphan_mask = ~gdf_fire_soil['point_index'].isin(matched_indices)
gdf_orphans = gdf_fire_soil[orphan_mask].copy()
n_orphans = len(gdf_orphans)
print(f"   📊 Points sans polygone : {n_orphans:,} ({n_orphans/len(gdf_fire_soil)*100:.2f}%)")

if n_orphans == 0:
    print(f"   ✅ Tous les points sont dans des polygones !")
    gdf_final = gdf_matched.copy()
else:
    print(f"   🔧 Attribution des points orphelins via DISTANCE MINIMALE")

# ============================================================================
# ÉTAPE 6 : ATTRIBUTION DES POINTS ORPHELINS AU POLYGONE LE PLUS PROCHE
# ============================================================================
print("\n[6/8] Attribution des points orphelins au polygone le plus proche...")

gdf_orphans_enriched = gdf_orphans.copy()
landcover_cols_to_copy = [col for col in gdf_landcover.columns if col != 'geometry']

for idx, point in gdf_orphans_enriched.iterrows():
    distances = gdf_landcover.geometry.distance(point.geometry)
    closest_idx = distances.idxmin()
    # Copier les colonnes LANDCOVER
    for col in landcover_cols_to_copy:
        gdf_orphans_enriched.at[idx, col] = gdf_landcover.at[closest_idx, col]

print(f"      ✓ {len(gdf_orphans_enriched):,} points orphelins enrichis")

# ============================================================================
# ÉTAPE 7 : FUSIONNER TOUS LES POINTS
# ============================================================================
print("\n[7/8] Fusion des points matchés et orphelins...")

common_cols = list(set(gdf_matched.columns) & set(gdf_orphans_enriched.columns))
gdf_final = pd.concat([
    gdf_matched[common_cols],
    gdf_orphans_enriched[common_cols]
], ignore_index=True)

print(f"      ✓ Dataset final : {len(gdf_final):,} points")

# ============================================================================
# ÉTAPE 8 : RÉORGANISATION ET SAUVEGARDE
# ============================================================================
print("\n[8/8] Réorganisation des colonnes et sauvegarde...")

cols_to_drop = ['index_right', 'point_index']  # suppression des colonnes techniques
gdf_final = gdf_final.drop(columns=[col for col in cols_to_drop if col in gdf_final.columns])

# Réorganiser colonnes
all_columns = list(gdf_final.columns)
base_cols = ['latitude', 'longitude']
target_col = ['class']
geo_col = ['geometry']
feature_cols = [col for col in all_columns if col not in base_cols + target_col + geo_col]
new_order = base_cols + feature_cols + target_col + geo_col
gdf_final = gdf_final[new_order]

# Conversion en GeoDataFrame
gdf_final = gpd.GeoDataFrame(gdf_final, geometry='geometry', crs='EPSG:4326')

# Sauvegarde
csv_output = os.path.join(output_dir, "fire_soil_landcover_merged_distance_min.csv")
df_csv = pd.DataFrame(gdf_final.drop(columns=['geometry']))
df_csv.to_csv(csv_output, index=False, encoding='utf-8-sig')

gpkg_output = os.path.join(output_dir, "fire_soil_landcover_merged_distance_min.gpkg")
gdf_final.to_file(gpkg_output, driver='GPKG')

try:
    excel_output = os.path.join(output_dir, "fire_soil_landcover_merged_distance_min.xlsx")
    df_csv.to_excel(excel_output, index=False, engine='openpyxl')
except Exception as e:
    print(f"⚠️ Excel non créé : {e}")

print("\n✅ FUSION FIRE+SOIL+LANDCOVER TERMINÉE AVEC SUCCÈS")


FUSION SPATIALE : FIRE+SOIL+LANDCOVER (DISTANCE MINIMALE POUR POINTS ORPHELINS)

[1/8] Chargement des données...
   📍 Chargement du dataset FIRE+SOIL...
      ✓ Fire+Soil chargé : 140,165 points

   🌳 Chargement du dataset LANDCOVER...
      ✓ Landcover chargé : 438,513 polygones

[2/8] Conversion en GeoDataFrames...
   📍 Création des points FIRE+SOIL...
      ✓ 140,165 points créés
   🌳 Conversion des polygones LANDCOVER...
      ✓ 438,513 polygones créés

[3/8] Jointure spatiale (intersects) pour points dans polygones...
      ✓ Points matchés : 139,479 correspondances

[4/8] Gestion des doublons...
   ✅ Aucun doublon détecté

[5/8] Identification des points orphelins...
   📊 Points sans polygone : 686 (0.49%)
   🔧 Attribution des points orphelins via DISTANCE MINIMALE

[6/8] Attribution des points orphelins au polygone le plus proche...
      ✓ 686 points orphelins enrichis

[7/8] Fusion des points matchés et orphelins...
      ✓ Dataset final : 140,165 points

[8/8] Réorganisation 

In [2]:
import os
import pandas as pd

# ============================================================================
# SUPPRESSION DES COLONNES DE MÉTADONNÉES
# ============================================================================
print("=" * 80)
print("SUPPRESSION DES COLONNES DE MÉTADONNÉES")
print("=" * 80)

# Chemin du fichier
base_path = r"C:\Users\pc\Desktop\DM\datasets"
input_file = os.path.join(base_path, "merged_fire_soil_landcover_distance_min", "fire_soil_landcover_merged_distance_min.csv")
output_file = os.path.join(base_path, "merged_fire_soil_landcover_distance_min", "fire_soil_landcover_merged_clean.csv")

# Colonnes à supprimer
cols_to_drop = ['knn_distance_km', 'knn_distance', 'assigned_method']

# ============================================================================
# CHARGEMENT ET TRAITEMENT
# ============================================================================
print("\n[1/3] Chargement du fichier...")
df = pd.read_csv(input_file)
print(f"   ✓ Fichier chargé : {len(df):,} lignes × {len(df.columns)} colonnes")

print("\n[2/3] Suppression des colonnes de métadonnées...")

# Vérifier quelles colonnes existent
existing_cols_to_drop = [col for col in cols_to_drop if col in df.columns]
missing_cols = [col for col in cols_to_drop if col not in df.columns]

if len(existing_cols_to_drop) > 0:
    print(f"\n   📋 Colonnes à supprimer :")
    for col in existing_cols_to_drop:
        print(f"      • {col}")
    
    # Supprimer les colonnes
    df = df.drop(columns=existing_cols_to_drop)
    
    print(f"\n   ✓ {len(existing_cols_to_drop)} colonnes supprimées")
else:
    print("   ℹ️  Aucune des colonnes spécifiées n'existe dans le fichier")

if len(missing_cols) > 0:
    print(f"\n   ⚠️  Colonnes non trouvées (déjà absentes) :")
    for col in missing_cols:
        print(f"      • {col}")

print(f"\n   📊 Nouveau dataset : {len(df):,} lignes × {len(df.columns)} colonnes")

# ============================================================================
# SAUVEGARDE
# ============================================================================
print("\n[3/3] Sauvegarde du fichier nettoyé...")

# Sauvegarder en CSV
df.to_csv(output_file, index=False, encoding='utf-8-sig')
print(f"   ✓ CSV sauvegardé : {output_file}")

# Optionnel : Sauvegarder en Excel
try:
    excel_output = output_file.replace('.csv', '.xlsx')
    df.to_excel(excel_output, index=False, engine='openpyxl')
    print(f"   ✓ Excel sauvegardé : {excel_output}")
except Exception as e:
    print(f"   ⚠️  Excel non créé : {e}")

# ============================================================================
# RAPPORT FINAL
# ============================================================================
print("\n" + "=" * 80)
print("✅ NETTOYAGE TERMINÉ")
print("=" * 80)

print(f"\n📊 RÉSUMÉ :")
print(f"   • Colonnes supprimées : {len(existing_cols_to_drop)}")
print(f"   • Colonnes restantes  : {len(df.columns)}")
print(f"   • Lignes conservées   : {len(df):,}")

print(f"\n📋 COLONNES FINALES ({len(df.columns)}) :")
for i, col in enumerate(df.columns, 1):
    print(f"   {i:2d}. {col}")

print(f"\n👀 APERÇU DES DONNÉES (5 premières lignes) :")
print("=" * 80)
display_cols = df.columns[:5].tolist()  # Afficher les 5 premières colonnes
print(df[display_cols].head())

print("\n" + "=" * 80)
print("🚀 FICHIER NETTOYÉ PRÊT À UTILISER !")
print("=" * 80)

SUPPRESSION DES COLONNES DE MÉTADONNÉES

[1/3] Chargement du fichier...
   ✓ Fichier chargé : 140,165 lignes × 16 colonnes

[2/3] Suppression des colonnes de métadonnées...

   📋 Colonnes à supprimer :
      • knn_distance_km
      • knn_distance
      • assigned_method

   ✓ 3 colonnes supprimées

   📊 Nouveau dataset : 140,165 lignes × 13 colonnes

[3/3] Sauvegarde du fichier nettoyé...
   ✓ CSV sauvegardé : C:\Users\pc\Desktop\DM\datasets\merged_fire_soil_landcover_distance_min\fire_soil_landcover_merged_clean.csv
   ✓ Excel sauvegardé : C:\Users\pc\Desktop\DM\datasets\merged_fire_soil_landcover_distance_min\fire_soil_landcover_merged_clean.xlsx

✅ NETTOYAGE TERMINÉ

📊 RÉSUMÉ :
   • Colonnes supprimées : 3
   • Colonnes restantes  : 13
   • Lignes conservées   : 140,165

📋 COLONNES FINALES (13) :
    1. latitude
    2. longitude
    3. TEB_log
    4. REF_BULK_log
    5. CEC_CLAY
    6. GYPSUM_log
    7. TEXTURE_SOTER_encoded
    8. ORG_CARBON_log
    9. LCCCODE_encoded
   10. ELEC_C